##### Imports

In [ ]:
# Parameters
top = 10  # default value

In [1]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.feature_selection \
        import mutual_info_regression
        
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [3]:
from pathlib import Path
import pandas as pd
from filelock import FileLock
import dask.dataframe as dd

# Define the path
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')
lock_path = "/tmp/assets_h5_file.lock"  # Lock path
top = 250
# Lock the file and retrieve the data
with FileLock(lock_path):
    # Use Dask to directly read the HDF5 files
    alpha_101_data_full = dd.read_hdf(DATA_STORE, \
        key=f'factor/top{top}_dataset_alpha101').compute()
    ta_data_full = dd.read_hdf(DATA_STORE, \
        key=f'factor/top{top}_dataset_with_TA').compute()
    beta_proxy_data_full = dd.read_hdf(DATA_STORE, \
        key=f'factor/top{top}_dataset_with_rolling_beta_size_proxy').compute()
    # with pd.HDFStore(DATA_STORE) as store:
    #     alpha_101_data_full = store['factors/alpha_101']
    #     common_data_full = store['factors/common']


# end_date = pd.Timestamp('2023-08-11')
# start_date = end_date - pd.tseries.offsets.BDay(5)

# print(start_date)


# alpha_101_data = alpha_101_data_full.loc[
#     (alpha_101_data_full.index.get_level_values("date") >= start_date) &
#     (alpha_101_data_full.index.get_level_values("date") <= end_date)
# ]

# common_data = common_data_full.loc[
#     (common_data_full.index.get_level_values("date") >= start_date) &
#     (common_data_full.index.get_level_values("date") <= end_date)
# ]

# del alpha_101_data_full
# del common_data_full

In [ ]:
# Prepare an empty list to collect processed chunks
processed_data_list = []

# Given that `common_data` is the larger DataFrame, we'll chunk through that.
# Define a chunk size based on your system's memory; you can adjust it as needed.
CHUNK_SIZE = 100000

# Create function to chunk through DataFrame
def chunker(seq, size):
    return (seq[pos: pos + size] for pos in range(0, len(seq), size))

# Chunk through `common_data` based on the index
for start in range(0, common_data.shape[0], CHUNK_SIZE):
    end = start + CHUNK_SIZE
    
    # Selecting the current chunk
    common_data_chunk = common_data.iloc[start:end]
    
    # Extract unique tickers and dates from the current chunk
    tickers_in_chunk = common_data_chunk.index.get_level_values('ticker').unique()
    dates_in_chunk = common_data_chunk.index.get_level_values('date').unique()
    
    # Filter alpha_101_data based on tickers and dates in the current chunk
    filtered_alpha_101_data = alpha_101_data[
        alpha_101_data.index.get_level_values('ticker').isin(tickers_in_chunk) &
        alpha_101_data.index.get_level_values('date').isin(dates_in_chunk)
    ]
    
    # Merge current chunk with filtered alpha_101_data
    merged_chunk = common_data_chunk.merge(
        filtered_alpha_101_data, left_index=True, right_index=True, how='inner', suffixes=('', '_y')
    )

    # Drop any columns that end with "_y" suffix and "_x" (just in case, though "_x" might not appear in this context)
    merged_chunk = merged_chunk.drop(columns=[col for col in merged_chunk if col.endswith(('_y', '_x'))])

    processed_data_list.append(merged_chunk)

# Concatenate all processed chunks
final_data = pd.concat(processed_data_list)

print(f"Shape of the final combined data: {final_data.shape}")
print("Processing completed.")

del processed_data_list

Shape of the final combined data: (3105, 624)
Processing completed.


In [ ]:
# Rename columns
final_data.columns = [f'FEATURE_{col.upper()}' if 'ret_fwd' not in \
    col else f'TARGET_{col.upper()}' for col in final_data.columns]

In [ ]:
ret_fwd_columns = [col for col in final_data.columns if col.startswith('TARGET_RET_FWD')]

if ret_fwd_columns:
    print(f"Columns that start with 'TARGET_RET_FWD': {ret_fwd_columns}")
else:
    print("No columns start with 'TARGET_RET_FWD'.")

No columns start with 'TARGET_RET_FWD'.


In [ ]:
from utils import save_to_hdf

# Example usage
FILE_PATH = "/home/sayem/Desktop/Project/data/dataset.h5"
# Define key name
KEY_NAME_PREFIX = f'data/YEAR'
_ = save_to_hdf(final_data, FILE_PATH, KEY_NAME_PREFIX)
print(f'key is: {_}')

key is: data_YEAR_2023_08_04_to_2023_08_11


In [ ]:
import dask.dataframe as dd
import pandas as pd

FILE_PATH = "/home/sayem/Desktop/Project/data/dataset.h5"

# Extract all keys from the HDF5 file
with pd.HDFStore(FILE_PATH) as store:
    available_keys = [key for key in store.keys() if key.startswith('/data_YEAR_')]

# Dictionary to store datasets corresponding to each key
datasets = {}

FILE_NAME = available_keys[0]

# for FILE_NAME in available_keys:
# Read the dataset using Dask for each key
data = dd.read_hdf(FILE_PATH, FILE_NAME)
    # Compute the result (this will load data into memory) 
    # and store it in the datasets dictionary with the key as the identifier
result = data.compute()
# break

# Now, the datasets dictionary contains data for each key.

In [ ]:
result

feature_FEATURE_OPEN  feature_FEATURE_HIGH  \
ticker date                                                     
AA     2023-08-04             34.549999             34.930000   
       2023-08-07             33.900002             34.060001   
       2023-08-08             33.020000             33.349998   
       2023-08-09             33.480000             34.090000   
       2023-08-10             33.970001             34.000000   
...                                 ...                   ...   
ZTS    2023-08-07            180.190002            181.880005   
       2023-08-08            182.149994            190.539993   
       2023-08-09            189.940002            190.949997   
       2023-08-10            190.000000            191.824997   
       2023-08-11            188.479996            191.600006   

                   feature_FEATURE_LOW  feature_FEATURE_CLOSE  \
ticker date                                                     
AA     2023-08-04            33.755001              33.950001   
       2023-08-07            33.220001              34.060001   
       2023-08-08            32.680000              33.230000   
       2023-08-09            33.029999              33.619999   
       2023-08-10            32.549999              32.880001   
...                                ...                    ...   
ZTS    2023-08-07           180.029999             180.690002   
       2023-08-08           177.250000             189.300003   
       2023-08-09           186.964996             189.350006   
       2023-08-10           187.729996             189.100006   
       2023-08-11           188.179993             190.830002   

                   feature_FEATURE_VOLUME  feature_FEATURE_MARKET_CAP  \
ticker date                                                             
AA     2023-08-04               3820122.0                5.703249e+09   
       2023-08-07               3520608.0                5.703249e+09   
       2023-08-08               5366294.0                5.703249e+09   
       2023-08-09               4132679.0                5.703249e+09   
       2023-08-10               4810865.0                5.703249e+09   
...                                   ...                         ...   
ZTS    2023-08-07               1941699.0                8.818486e+10   
       2023-08-08               4117986.0                8.818486e+10   
       2023-08-09               2270030.0                8.818486e+10   
       2023-08-10               2348639.0                8.818486e+10   
       2023-08-11               1742931.0                8.818486e+10   

                   feature_FEATURE_SECTOR  feature_FEATURE_RET_FRAC_ORDER  \
ticker date                                                                 
AA     2023-08-04                     4.0                        4.251619   
       2023-08-07                     4.0                        4.512142   
       2023-08-08                     4.0                        3.725083   
       2023-08-09                     4.0                        4.353477   
       2023-08-10                     4.0                        3.625935   
...                                   ...                             ...   
ZTS    2023-08-07                     2.0                       16.669207   
       2023-08-08                     2.0                       24.922548   
       2023-08-09                     2.0                       21.726665   
       2023-08-10                     2.0                       20.455498   
       2023-08-11                     2.0                       21.730034   

                   feature_FEATURE_RET_01D  feature_FEATURE_RET_02D  \
ticker date                                                           
AA     2023-08-04                -0.015942                 0.000589   
       2023-08-07                 0.003240                -0.012754   
       2023-08-08                -0.024369                -0.021208   
       2023-08-09   

In [1]:
results.info()

NameError: name 'results' is not defined

In [ ]:
from utils import clear_large_vars
clear_large_vars()